# Transformer
---------------------------
~~이름도 멋진 트랜스포머~~
### Multilayer Transformer Decoder
트랜스포머는 encoder-decoder로 이루어져 있다. 인코더와 디코더는 마스킹의 차이를 빼고는 동일한 모델을 사용한 레이어의 적층구조이다. 'Improving Language Understanding by Generative Pre-Training' 은 Transformer의 decoder 부분만을 이용한다. 구조는 Masked multi-headed self attention과 Normalization layer, Position-wise FFN의 조합과 residual connection을 사용한 decoder를 12개를 쌓는 방식으로 이루어져있다. 이 글에서는 Masked multi-headed self attention을, self attention, mult-headed 부분을 나누어서 점차적으로 설명하려고 한다.

### Self Attention
Self attention. 말 그대로 input text vector들의 얽히고 섥힌 관계를 구하는 단계이다. 수 많은 블로그에서 설명이 되어있듯이, RNN 등 auto-regressive model 들의 단점은 병렬연산이 힘들다는 점부터 여러 단점이 있지만, 그 중 하나는 sequence 내에 time-step이 멀리 떨어진 token 들 간의 관계를 계산하는게 약하는 점이 있다. 이를 보완하기 위해 attention mechanism을 RNN과 같이 사용하는 등의 발전으로 breakthrough가 이루어졌지만, 최근 CNN계열의 language model이나, 'Attention is all you need!' 등의 attention only FFN 모델에 의한 방식이 SOTA 퍼포먼스를 보여주고 있다.
<br><br>
__연산 과정은 다음과 같다.__
1. __Key/Value/Query matrix__<br>
인풋인 Text embedding matrix를 tranform해 Key, Value, Query matrix를 만든다.<br>
이 때, 세개의 독립적인 weight matrix를 embedding mat에 곱해 K/V/Q를 만들기 때문에 각 weight matrix는 독립적으로 학습된다.<br>
<br>
2. __Score__<br>
K/V/Q를 구했다면, 다음 step으로 단어간의 score를 구해야 한다.<br>
Score는 총 N by N 의 갯수만큼 구해야 한다. 이 점수는 Q×Transpose(K) 를 통해서, Q1×T(K1), Q1×T(K2), Q1×(K3).... 와 같이 특정 토큰의 Qeury와 전체 단어의 Key의 dot product를 통해서 구한다.<br>
<br>
3. __Divide and Softmax__<br>
점수를 구했다면 Dimension size의 제곱근으로 나누어준다. 저자는 이를 통해 gradient descent가 더 안정적이 된다고 주장한다.<br>
이렇게 구한 점수값 들에 softmax를 해준다<br>
<br>
4. __Weighted Values__<br>
Softmax를 통해서 구해진 값에 V를 곱해준다. 이를 통해 특정 토큰의 Value 값이 sentence 내의 다른 토큰들의 비중에 의해 가중된 값을 가지게 된다<br>
<br>
5. __Attention__<br>
Weighted value들 전체를 더해준다
<img src="https://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" alt="Drawing" style="width: 600px;"/>


### Multi-headed Self Attention
Multi-headed self attention은 그냥 self attention을 여러개 병렬적으로 수행하는 것이다. 하나의 self attention matrix output은 토큰들의 관계와 의미를 나타내는 하나의 관점이라 생각할 수 있다. 이런 self attention을 여러개 가진다는 것은 수 개의 다른 관점을 가지는 효과를 가진다.<br>
<br>
__위의 Self attention에 아래 과정을 추가한다.__<br>
1. __Multiple matrices__<br>
원하는 갯수만큼의 self attention 매트릭스를 구한다<br>
<br>
2. __Concatenation__<br>
Self attention matrix Z1, Z2, ... Zn을 row concatenation을 한다.<br>
<br>
3. __Weight all__<br>
Concatenation이 된 Z 매트릭스를 weight matrix W로 곱해줘 다시 self attention matrix 한개의 사이즈로 transform해준다.
<img src="https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png" alt="Drawing" style="width: 600px;"/>

위 그림들은 [illustrated transformer](https://jalammar.github.io/illustrated-transformer/)가 출처이며, 위 글은 Transformer와 self attention을 이해하기에 매우 좋은 글이어서 꼭 읽어보기를 추천한다

### Code explanation
masked multi attention 후에, normalization, FFN, 그리고 다시 normalization을 하면 한 레이어가 끝난다. 인풋과 아웃풋의 사이즈는 동일

먼저 인풋 매트릭스를 k, v, q로 나누어준다. conv1d를 사용

각 k,v,q 매트릭스를 헤드의 수만큼 나눈다. N_HEAD = 12 라면, 12개의 변수로 쪼개는 것이 아니라 reshape을 통해 차원을 추가한다

그 후 self attention의 score를 구하는 과정, key와 곱해 비중을 구하는 과정 등등을 수행한다